<a href="https://colab.research.google.com/github/navneetkrc/Deep-Learning-Experiments-implemented-using-Google-Colab/blob/master/Text_Summarization_via_CLI_using_Ollama_with_Google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 - Install Required Libraries:
!pip install streamlit PyMuPDF4LLM pandas openpyxl pyngrok requests  # Added 'requests' for Ollama API calls

# Install Ollama (using a convenient installation script)
!curl -fsSL https://ollama.com/install.sh | sh

# Install and setup ngrok (as in your original code)
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
# Cell 2 - Import Libraries and Set Up Environment:
import streamlit as st
import fitz  # PyMuPDF4LLM
import pandas as pd
import os
import time
import requests  # For making requests to the Ollama API
from google.colab import files

from google.colab import userdata
# Remove Groq API key setup
# os.environ["GROQ_API_KEY"] = userdata.get('groq_colab_key') #krch
# os.environ["GROQ_API_KEY"] = userdata.get('groq_604779')
os.environ["NGROK_AUTH_TOKEN2"] = userdata.get('NGROK_AUTH_TOKEN')

# Get ngrok auth token from environment
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
!ngrok authtoken {ngrok_token}

# Start Ollama in the background (replace 'model_one' and 'model_two' with your actual model names)
!nohup ollama serve > ollama.log 2>&1 &
!ollama pull gemma2  # Example - replace with your actual model names
!ollama pull llama3.2 # Example - replace with your actual model names
!ollama pull smallthinker # Example - replace with your actual model names
!ollama pull qwen2.5 # Example - replace with your actual model names
time.sleep(10) # Give Ollama time to start

In [ ]:
# Start Ollama in the background (replace 'model_one' and 'model_two' with your actual model names)
!nohup ollama serve > ollama.log 2>&1 &
# !ollama pull gemma2  # Example - replace with your actual model names
# !ollama pull llama3.2 # Example - replace with your actual model names
# !ollama pull smallthinker # Example - replace with your actual model names
# !ollama pull qwen2.5 # Example - replace with your actual model names
time.sleep(10) # Give Ollama time to start

In [ ]:
# Install required packages
!pip install streamlit langchain-community

In [ ]:
##1 upload
# %%writefile app.py
# import streamlit as st
# import pandas as pd
# from langchain_community.llms import Ollama
# import io
# import requests

# # Configure page settings
# st.set_page_config(page_title="Multi-Model Text Summarizer", page_icon="📝")

# # Define constants
# OLLAMA_BASE_URL = "http://localhost:11434/"
# # MODELS = ["gemma2", "llama3.2", "smallthinker", "qwen2.5"]
# MODELS = ["gemma2"]
# def check_ollama_server():
#     """Check if Ollama server is running"""
#     try:
#         requests.get(f"{OLLAMA_BASE_URL}")
#         return True
#     except requests.exceptions.ConnectionError:
#         return False

# def verify_model(model_name):
#     """Verify if a model is available in Ollama"""
#     try:
#         response = requests.post(
#             f"{OLLAMA_BASE_URL}api/show",
#             json={"name": model_name}
#         )
#         return response.status_code == 200
#     except requests.exceptions.ConnectionError:
#         return False

# def create_summary(content, model_name):
#     """
#     Creates a summary using specified Ollama model
#     Args:
#         content (str): Text content to summarize
#         model_name (str): Name of the model to use
#     Returns:
#         str: Summarized text
#     """
#     try:
#         ollama = Ollama(base_url=OLLAMA_BASE_URL, model=model_name)
#         prompt = f"""
#                 Summarize the following text:
#                 {content.strip()}
#                 Summarize the following Samsung support document in a bullet point list. The summary should be no more than 5 bullet points and capture the most essential information.
#                 """
#         response = ollama.invoke(prompt)
#         return response.strip()
#     except Exception as e:
#         return f"Error with {model_name}: {str(e)}"

# def process_files(uploaded_files):
#     """
#     Process multiple files and generate summaries using all models
#     Args:
#         uploaded_files: List of uploaded file objects
#     Returns:
#         pd.DataFrame: DataFrame containing summaries from all models
#     """
#     results = []

#     # Create progress bar
#     progress_bar = st.progress(0)
#     total_files = len(uploaded_files)

#     for i, file in enumerate(uploaded_files):
#         # Read file content
#         content = file.read().decode('utf-8')

#         # Get summaries from each model
#         summaries = {model: create_summary(content, model) for model in MODELS}

#         # Add to results
#         result = {"file_name": file.name}
#         result.update(summaries)
#         results.append(result)

#         # Update progress bar
#         progress_bar.progress((i + 1) / total_files)

#     return pd.DataFrame(results)

# def main():
#     st.title("📝 Multi-Model Text Summarizer")
#     st.markdown("Upload multiple text files to get summaries from different models.")

#     # Check server status
#     server_status = check_ollama_server()
#     if server_status:
#         st.success("✅ Ollama server is running")
#     else:
#         st.error("❌ Ollama server is not running. Please start it with 'ollama serve'")
#         return

#     # Check model status
#     st.subheader("Model Status:")
#     models_available = True
#     for model in MODELS:
#         if verify_model(model):
#             st.success(f"✅ {model} is available")
#         else:
#             st.error(f"❌ {model} is not available. Please pull it with 'ollama pull {model}'")
#             models_available = False

#     if not models_available:
#         st.warning("Please make sure all models are available before proceeding.")
#         return

#     # File upload section
#     uploaded_files = st.file_uploader(
#         "Upload text files",
#         type=['txt'],
#         accept_multiple_files=True
#     )

#     if uploaded_files:
#         # Display file names
#         st.write("Uploaded files:")
#         for file in uploaded_files:
#             st.write(f"- {file.name}")

#         if st.button("Generate Summaries"):
#             with st.spinner("Processing files..."):
#                 # Process files and get DataFrame
#                 df = process_files(uploaded_files)

#                 # Save to Excel
#                 excel_buffer = io.BytesIO()
#                 df.to_excel(excel_buffer, index=False)
#                 excel_buffer.seek(0)

#                 # Display results
#                 st.success("Processing complete!")
#                 st.subheader("Preview of Results:")
#                 st.dataframe(df)

#                 # Add download button for Excel file
#                 st.download_button(
#                     label="Download Excel File",
#                     data=excel_buffer,
#                     file_name="summaries.xlsx",
#                     mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
#                 )

# if __name__ == "__main__":
#     main()

Writing app.py


In [ ]:

# %%writefile app.py

# import streamlit as st
# from langchain_community.llms import Ollama

# # Configure page settings
# st.set_page_config(page_title="Text Summarizer", page_icon="📝")

# # Define constants
# OLLAMA_BASE_URL = "http://localhost:11434/"
# OLLAMA_MODEL_NAME = "gemma2"

# def create_summary(content):
#     """
#     Creates a summary using Ollama model
#     Args:
#         content (str): Text content to summarize
#     Returns:
#         str: Summarized text
#     """
#     try:
#         # Creating Ollama client
#         ollama = Ollama(base_url=OLLAMA_BASE_URL, model=OLLAMA_MODEL_NAME)

#         # Improved prompt design
#         prompt = f"""
#                 Summarize the following text:
#                 {content.strip()}
#                 Please provide a concise summary of the key points in the text.
#                 """

#         # Making the API call
#         response = ollama.invoke(prompt)
#         return response.strip()
#     except Exception as e:
#         return f"Error: An error occurred during summarization: {e}"

# def main():
#     # Add title and description
#     st.title("📝 Text Summarizer")
#     st.markdown("Upload a text file or enter text directly to get a summary.")

#     # Create tabs for different input methods
#     tab1, tab2 = st.tabs(["Enter Text", "Upload File"])

#     with tab1:
#         # Text input
#         text_input = st.text_area("Enter your text here:", height=200)
#         if st.button("Summarize Text", key="summarize_text"):
#             if text_input:
#                 with st.spinner("Generating summary..."):
#                     summary = create_summary(text_input)
#                 st.subheader("Summary:")
#                 st.write(summary)
#             else:
#                 st.error("Please enter some text to summarize.")

#     with tab2:
#         # File upload
#         uploaded_file = st.file_uploader("Choose a text file", type=['txt'])
#         if uploaded_file is not None:
#             content = uploaded_file.read().decode()
#             st.text_area("File content:", content, height=200)
#             if st.button("Summarize File", key="summarize_file"):
#                 with st.spinner("Generating summary..."):
#                     summary = create_summary(content)
#                 st.subheader("Summary:")
#                 st.write(summary)

# if __name__ == "__main__":
#     main()

Writing app.py


In [ ]:
%%writefile app.py
import streamlit as st
from langchain_community.llms import Ollama
import pandas as pd
import io
import requests

# Configure page settings
st.set_page_config(page_title="Text Summarizer", page_icon="📝")

# Define constants
OLLAMA_BASE_URL = "http://localhost:11434/"
MODELS = ["gemma2", "llama3.2", "smallthinker", "qwen2.5"]
# MODELS = ["gemma:2b", "llama2", "smallthinker", "qwen:2"]
def check_ollama_server():
    """Check if Ollama server is running"""
    try:
        requests.get(f"{OLLAMA_BASE_URL}")
        return True
    except requests.exceptions.ConnectionError:
        return False

def verify_model(model_name):
    """Verify if a model is available in Ollama"""
    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}api/show",
            json={"name": model_name}
        )
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

def get_prompt(prompt_style, content):
    """Get the appropriate prompt template"""
    prompts = {
        "Action-Focused": {
            "name": "Action-Focused",
            "template": f"""
            Summarize the following Samsung support document into a brief bullet point list outlining the key steps, solutions, or important information a customer needs to know quickly. Focus on actionable advice and problem resolution.
            Text:
            {content.strip()}
            """
        },
        "Structured-Technical": {
            "name": "Structured-Technical",
            "template": f"""
            Summarize the following Samsung support document into a bullet point list covering these aspects:
            ***Problem/Issue Addressed:** (What is this document about?)
            ***Key Steps/Solutions:** (What are the main actions or solutions?)
            ***Important Notes/Cautions:** (Are there any crucial warnings or things to remember?)
            Text:
            {content.strip()}
            """
        },
        "Five-Point-Essential": {
            "name": "Five-Point-Essential",
            "template": f"""
            Summarize the following Samsung support document in a bullet point list. The summary should be no more than 5 bullet points and capture the most essential information.
            Text:
            {content.strip()}
            """
        },
        "Quick-Resolution": {
            "name": "Quick-Resolution",
            "template": f"""
            Create an ultra-concise summary of this Samsung support document using this specific format:
            • Issue: [One-line problem description]
            • Fix: [Main solution in 1-2 steps]
            • Note: [One critical warning or tip]

            Keep each bullet to 10 words or less. Focus only on the most critical information that resolves the issue immediately.

            Text:
            {content.strip()}
            """
        }
    }
    return prompts[prompt_style]["template"]

def create_summary(content, model_name, prompt_style):
    """
    Creates a summary using specified Ollama model and prompt style
    """
    try:
        ollama = Ollama(base_url=OLLAMA_BASE_URL, model=model_name)
        prompt = get_prompt(prompt_style, content)
        response = ollama.invoke(prompt)
        return response.strip()
    except Exception as e:
        return f"Error: An error occurred during summarization: {e}"

def process_files(uploaded_files, selected_model, prompt_style):
    """Process multiple files and return results as DataFrame"""
    results = []

    with st.spinner("Processing files..."):
        for file in uploaded_files:
            content = file.read().decode()
            summary = create_summary(content, selected_model, prompt_style)

            results.append({
                "file_name": file.name,
                "model_name": selected_model,
                "prompt_style": prompt_style,
                "summary": summary
            })

    return pd.DataFrame(results)

def main():
    st.title("📝 Customizable Text Summarizer")
    st.markdown("Choose your model, prompt style, and upload files for summarization.")

    # Check Ollama server
    if not check_ollama_server():
        st.error("❌ Ollama server is not running. Please start it with 'ollama serve'")
        return

    # Model selection
    selected_model = st.selectbox(
        "Select Model",
        MODELS,
        help="Choose the AI model for summarization"
    )

    # Verify selected model
    if not verify_model(selected_model):
        st.error(f"❌ {selected_model} is not available. Please pull it with 'ollama pull {selected_model}'")
        return

    # Prompt style selection
    prompt_style = st.selectbox(
        "Select Prompt Style",
        [
            "Action-Focused",
            "Structured-Technical",
            "Five-Point-Essential",
            "Quick-Resolution"
        ],
        help="""
        Action-Focused: Step-by-step solution guide
        Structured-Technical: Comprehensive problem analysis
        Five-Point-Essential: Limited key points
        Quick-Resolution: Ultra-concise fix guide
        """
    )

    # Create tabs for different input methods
    tab1, tab2 = st.tabs(["Enter Text", "Upload Files"])

    with tab1:
        text_input = st.text_area("Enter your text here:", height=200)
        if st.button("Summarize Text", key="summarize_text"):
            if text_input:
                with st.spinner("Generating summary..."):
                    summary = create_summary(text_input, selected_model, prompt_style)
                st.subheader("Summary:")
                st.write(summary)
            else:
                st.error("Please enter some text to summarize.")

    with tab2:
        uploaded_files = st.file_uploader(
            "Choose text files",
            type=['txt'],
            accept_multiple_files=True
        )

        if uploaded_files:
            if st.button("Process Files", key="process_files"):
                # Process files and create DataFrame
                df = process_files(uploaded_files, selected_model, prompt_style)

                # Create Excel file
                excel_buffer = io.BytesIO()
                df.to_excel(excel_buffer, index=False)
                excel_buffer.seek(0)

                # Show results preview
                st.subheader("Results Preview:")
                st.dataframe(df)

                # Add download button
                st.download_button(
                    label="Download Excel Report",
                    data=excel_buffer,
                    file_name="summaries.xlsx",
                    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
                )

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!ollama list

Error: could not connect to ollama app, is it running?


In [ ]:
from pyngrok import ngrok
import time

# Kill any existing Streamlit processes
!kill -9 $(pgrep streamlit) 2>/dev/null

#Uncomment this in case of reruns
# !nohup ollama serve > ollama.log 2>&1 &
# time.sleep(10)

# Start Streamlit
!streamlit run app.py &>/content/logs.txt &
time.sleep(5)

# Create ngrok tunnel with correct configuration
ngrok_tunnel = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print(f"Streamlit app URL: {ngrok_tunnel.public_url}")

# Keep the tunnel open
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Closing ngrok tunnel...")
    ngrok.kill()

Streamlit app URL: https://b73e-34-127-125-126.ngrok-free.app


In [ ]:
%%writefile summarizer.py
import argparse
import json
import requests
from typing import Optional
import sys

class OllamaSummarizer:
    def __init__(self, base_url: str = "http://localhost:11434", model: str = "qwen2:0.5b"):
        """Initialize the Ollama API client.

        Args:
            base_url (str): Base URL for Ollama API
            model (str): Model name to use for summarization
        """
        self.base_url = base_url.rstrip('/')
        self.model = model
        self.generate_url = f"{self.base_url}/api/generate"

    def check_api_status(self) -> bool:
        """Check if Ollama API is accessible."""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            return response.status_code == 200
        except requests.RequestException:
            return False

    def generate_summary(self, content: str) -> str:
        """Generate summary using Ollama API.

        Args:
            content (str): Text content to summarize

        Returns:
            str: Generated summary
        """
        prompt = f"""
        Summarize the following text:
        {content.strip()}
        Please provide a concise summary of the key points in the text.
        """

        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False
        }

        try:
            response = requests.post(self.generate_url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result.get('response', '').strip()
        except requests.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

def read_file(file_path: str) -> str:
    """Read content from a file.

    Args:
        file_path (str): Path to the file

    Returns:
        str: File content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except IOError as e:
        raise Exception(f"Error reading file: {str(e)}")

def main():
    parser = argparse.ArgumentParser(description="Text Summarizer using Ollama API")

    # Add arguments
    input_group = parser.add_mutually_exclusive_group(required=True)
    input_group.add_argument('-t', '--text', type=str, help="Text to summarize")
    input_group.add_argument('-f', '--file', type=str, help="Path to text file to summarize")

    parser.add_argument('--url', type=str,
                       default="http://localhost:11434",
                       help="Ollama API base URL")
    parser.add_argument('--model', type=str,
                       default="qwen2:0.5b",
                       help="Model name to use")

    args = parser.parse_args()

    try:
        # Initialize summarizer
        summarizer = OllamaSummarizer(base_url=args.url, model=args.model)

        # Check API status
        if not summarizer.check_api_status():
            print("Error: Cannot connect to Ollama API. Make sure the server is running.")
            sys.exit(1)

        # Get content from either text or file
        content = read_file(args.file) if args.file else args.text

        # Generate summary
        print("Generating summary...")
        summary = summarizer.generate_summary(content)

        print("\nSummary:")
        print("-" * 80)
        print(summary)
        print("-" * 80)

    except Exception as e:
        print(f"Error: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Overwriting summarizer.py


In [ ]:
!python summarizer.py -f /content/book.txt

Generating summary...
Error: API request failed: 404 Client Error: Not Found for url: http://localhost:11434/api/generate


In [ ]:
# Running ollama in background in Google colab

In [ ]:
import subprocess
import requests
import time
import json
from typing import Optional

class OllamaManager:
    def __init__(self, base_url: str = "http://localhost:11434"):
        self.base_url = base_url.rstrip('/')

    def install_ollama(self) -> bool:
        """Install Ollama if not already installed"""
        try:
            print("Installing Ollama...")
            subprocess.run("curl https://ollama.ai/install.sh | sh", shell=True, check=True)
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error installing Ollama: {e}")
            return False

    def start_server(self) -> bool:
        """Start Ollama server in background"""
        try:
            print("Starting Ollama server...")
            subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # Wait for server to start
            for _ in range(5):  # Try for 5 seconds
                time.sleep(1)
                try:
                    if self.check_server():
                        print("Ollama server started successfully!")
                        return True
                except:
                    continue
            return False
        except Exception as e:
            print(f"Error starting server: {e}")
            return False

    def check_server(self) -> bool:
        """Check if Ollama server is running"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            return response.status_code == 200
        except:
            return False

    def pull_model(self, model_name: str) -> bool:
        """Pull a model from Ollama"""
        try:
            print(f"Pulling model {model_name}...")
            subprocess.run(["ollama", "pull", model_name], check=True)
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error pulling model: {e}")
            return False

    def list_models(self) -> list:
        """List all available models"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            if response.status_code == 200:
                return [model['name'] for model in response.json()['models']]
            return []
        except:
            return []

    def generate_response(self, model: str, prompt: str) -> Optional[str]:
        """Generate response using specified model"""
        try:
            payload = {
                "model": model,
                "prompt": prompt,
                "stream": False
            }
            response = requests.post(f"{self.base_url}/api/generate", json=payload)
            if response.status_code == 200:
                return response.json().get('response', '')
            return None
        except Exception as e:
            print(f"Error generating response: {e}")
            return None

# Example usage
def main():
    # Initialize Ollama manager
    manager = OllamaManager()

    # Install and start Ollama
    if not manager.install_ollama():
        print("Failed to install Ollama")
        return

    if not manager.start_server():
        print("Failed to start Ollama server")
        return

    # Pull a model (e.g., gemma:2b)
    model_name = "gemma:2b"
    if not manager.pull_model(model_name):
        print(f"Failed to pull model {model_name}")
        return

    # List available models
    print("\nAvailable models:")
    models = manager.list_models()
    print(models)

    # Test generation
    prompt = "What is machine learning?"
    print(f"\nTesting with prompt: {prompt}")
    response = manager.generate_response(model_name, prompt)
    if response:
        print("\nResponse:")
        print(response)

if __name__ == "__main__":
    main()

Installing Ollama...
Starting Ollama server...
Ollama server started successfully!
Pulling model gemma:2b...

Available models:
['gemma:2b', 'gemma2:latest', 'llama3.2:latest']

Testing with prompt: What is machine learning?

Response:
Sure. Here's a definition of machine learning:

**Machine learning** is a subfield of artificial intelligence (AI) that allows machines to learn from data without explicit programming. It involves algorithms that can analyze data, identify patterns, and make predictions or decisions based on those patterns.

**Key concepts in machine learning:**

* **Data:** Large amounts of data are the foundation of machine learning.
* **Algorithms:** These are sets of rules or instructions that the algorithm follows to analyze and learn from data.
* **Learning:** The algorithm is trained on data to learn the underlying patterns.
* **Prediction:** Once trained, the algorithm can make predictions or decisions based on new data.
* **Evaluation:** The accuracy and perform

In [ ]:
# Create manager instance
manager = OllamaManager()

# Generate a response
response = manager.generate_response("gemma2", "Explain quantum computing")
print(response)

Sure. Here's a simplified explanation of quantum computing:

**What is it?**

Quantum computing is a new type of computing that leverages the principles of quantum mechanics to solve problems that are too complex for classical computers.

**Key principles:**

* **Qubits**: Unlike classical bits that are either 0 or 1, qubits can exist in a state called superposition, where they are both 0 and 1 at the same time.
* **Entanglement**: Qubits can also be entangled, meaning their states are correlated even when physically separated. This allows them to share information instantaneously.
* **Quantum algorithms**: These are algorithms specifically designed to run on quantum computers, harnessing these principles to solve problems much faster than classical algorithms.

**Benefits:**

* **Drug discovery**: Quantum computing can significantly accelerate drug discovery by simulating molecular interactions and chemical reactions.
* **Materials science**: It can help design new materials with desi

In [ ]:
# Pull a different model
manager.pull_model("gemma2")

# List available models
print(manager.list_models())

# Generate with new model
response = manager.generate_response("gemma2", "What is AI?")
print(response)

Pulling model gemma2...
['gemma2:latest', 'gemma:2b', 'llama3.2:latest']
AI, or **Artificial Intelligence**, is a broad field of computer science focused on creating intelligent agents—systems that can reason, learn, and act autonomously. 

Here's a breakdown:

**Key Characteristics of AI:**

* **Learning:** AI systems can analyze data and learn patterns, improving their performance over time.
* **Reasoning:** They can use logic to solve problems and make decisions.
* **Problem-solving:** AI can find solutions to complex issues by analyzing information and identifying optimal courses of action.
* **Perception:** Some AI systems can interpret sensory input like images, sound, and text.
* **Natural Language Understanding:**  AI can understand and respond to human language in a meaningful way.

**Types of AI:**

* **Narrow or Weak AI:** Designed for specific tasks, like playing chess or recommending products. (This is the most common type today)
* **General or Strong AI:** Hypothetical AI

In [ ]:
import argparse
import json
import requests
import sys
import subprocess
import time
from typing import Optional

class OllamaManager:
    def __init__(self, base_url: str = "http://localhost:11434"):
        self.base_url = base_url.rstrip('/')
        self.ensure_server_running()

    def ensure_server_running(self):
        """Ensure Ollama server is running, start if not"""
        if not self.check_server():
            self.start_server()

    def check_server(self) -> bool:
        """Check if Ollama server is running"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            return response.status_code == 200
        except:
            return False

    def start_server(self):
        """Start Ollama server"""
        try:
            print("Starting Ollama server...")
            subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # Wait for server to start
            for _ in range(10):  # Try for 10 seconds
                time.sleep(1)
                if self.check_server():
                    print("Ollama server started successfully!")
                    return
            raise Exception("Server failed to start within timeout")
        except Exception as e:
            raise Exception(f"Error starting server: {e}")

class OllamaSummarizer:
    def __init__(self, base_url: str = "http://localhost:11434", model: str = "qwen2:0.5b"):
        """Initialize the Ollama API client.

        Args:
            base_url (str): Base URL for Ollama API
            model (str): Model name to use for summarization
        """
        self.base_url = base_url.rstrip('/')
        self.model = model
        self.generate_url = f"{self.base_url}/api/generate"
        self.manager = OllamaManager(base_url)

    def generate_summary(self, content: str) -> str:
        """Generate summary using Ollama API.

        Args:
            content (str): Text content to summarize

        Returns:
            str: Generated summary
        """
        prompt = f"""
        Summarize the following text:
        {content.strip()}
        Please provide a concise summary of the key points in the text.
        """

        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False
        }

        try:
            response = requests.post(self.generate_url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result.get('response', '').strip()
        except requests.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

def read_file(file_path: str) -> str:
    """Read content from a file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except IOError as e:
        raise Exception(f"Error reading file: {str(e)}")

def main():
    parser = argparse.ArgumentParser(description="Text Summarizer using Ollama API")

    input_group = parser.add_mutually_exclusive_group(required=True)
    input_group.add_argument('-t', '--text', type=str, help="Text to summarize")
    input_group.add_argument('-f', '--file', type=str, help="Path to text file to summarize")

    parser.add_argument('--url', type=str,
                       default="http://localhost:11434",
                       help="Ollama API base URL")
    parser.add_argument('--model', type=str,
                       default="qwen2:0.5b",
                       help="Model name to use")

    args = parser.parse_args()

    try:
        # Initialize summarizer (this will also ensure server is running)
        summarizer = OllamaSummarizer(base_url=args.url, model=args.model)

        # Get content from either text or file
        content = read_file(args.file) if args.file else args.text

        # Generate summary
        print("Generating summary...")
        summary = summarizer.generate_summary(content)

        print("\nSummary:")
        print("-" * 80)
        print(summary)
        print("-" * 80)

    except Exception as e:
        print(f"Error: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Generating summary...
Error: API request failed: 404 Client Error: Not Found for url: http://localhost:11434/api/generate
Traceback (most recent call last):
  File "<ipython-input-21-2a53eb0911b8>", line 78, in generate_summary
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/generate

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-21-2a53eb0911b8>", line 117, in main
    summary = summarizer.generate_summary(content)
  File "<ipython-input-21-2a53eb0911b8>", line 82, in generate_summary
    raise Exception(f"API request failed: {str(e)}")
Exception: API request failed: 404 Client Error: Not Found for url: http://localhost:11434/api/generate

During handling of the above exception, 

TypeError: object of type 'NoneType' has no len()

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  52432      0 --:--:-- --:--:-- --:--:-- 52654
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama pull gemma2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff1d1fc78170... 100% ▕▏ 5.4 GB                         
pulling 109037bec39c... 100% ▕▏  136 B                         
pulling 097a36493f71... 100% ▕▏ 8.4 KB                         
pulling 2490e7468436... 100% ▕▏   65 B                         
pulling 10aa81da732e... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama list

NAME               ID              SIZE      MODIFIED               
gemma2:latest      ff02c3702f32    5.4 GB    Less than a second ago    
gemma:2b           b50d6c999e59    1.7 GB    About a minute ago        
llama3.2:latest    a80c4f17acd5    2.0 GB    5 minutes ago             


In [ ]:
%%writefile multiprompt_summarizer.py
import argparse
import json
import requests
import sys
import subprocess
import time
from typing import Optional, List, Dict
from dataclasses import dataclass

@dataclass
class SummaryPrompt:
    name: str
    template: str

class OllamaManager:
    def __init__(self, base_url: str = "http://localhost:11434"):
        self.base_url = base_url.rstrip('/')
        self.ensure_server_running()

    def ensure_server_running(self):
        """Ensure Ollama server is running, start if not"""
        if not self.check_server():
            self.start_server()

    def check_server(self) -> bool:
        """Check if Ollama server is running"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            return response.status_code == 200
        except:
            return False

    def start_server(self):
        """Start Ollama server"""
        try:
            print("Starting Ollama server...")
            subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            for _ in range(10):
                time.sleep(1)
                if self.check_server():
                    print("Ollama server started successfully!")
                    return
            raise Exception("Server failed to start within timeout")
        except Exception as e:
            raise Exception(f"Error starting server: {e}")

class MultiPromptSummarizer:
    def __init__(self, base_url: str = "http://localhost:11434", model: str = "qwen2:0.5b"):
        self.base_url = base_url.rstrip('/')
        self.model = model
        self.generate_url = f"{self.base_url}/api/generate"
        self.manager = OllamaManager(base_url)

        # Define the prompts
        self.prompts = [
            SummaryPrompt(
                "TLDR Style",
                """Provide a concise, bullet-point "TLDR" summary of the following text. Highlight the most crucial information a reader should understand at a glance.
                Text: {content}"""
            ),
            SummaryPrompt(
                "Structured Analysis",
                """Summarize the following text into a bullet point list covering these aspects:
                ***Problem/Issue Addressed:** (What is this document about?)
                ***Key Steps/Solutions:** (What are the main actions or solutions?)
                ***Important Notes/Cautions:** (Are there any crucial warnings or things to remember?)
                Text: {content}"""
            ),
            SummaryPrompt(
                "Concise 5-Point",
                """Summarize the following text in a bullet point list. The summary should be no more than 5 bullet points and capture the most essential information.
                Text: {content}"""
            ),
            SummaryPrompt(
                "Comprehensive TLDR",
                """Provide a comprehensive but clear summary of the following text, focusing on key points and actionable information. Format the response in clear sections.
                Text: {content}"""
            )
        ]

    def generate_summary(self, content: str, prompt_template: str) -> str:
        """Generate summary using Ollama API with specific prompt"""
        payload = {
            "model": self.model,
            "prompt": prompt_template.format(content=content.strip()),
            "stream": False
        }

        try:
            response = requests.post(self.generate_url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result.get('response', '').strip()
        except requests.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

    def summarize_all(self, content: str) -> Dict[str, str]:
        """Generate summaries using all prompts"""
        summaries = {}
        for prompt in self.prompts:
            print(f"\nGenerating {prompt.name} summary...")
            summary = self.generate_summary(content, prompt.template)
            summaries[prompt.name] = summary
        return summaries

def read_file(file_path: str) -> str:
    """Read content from a file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except IOError as e:
        raise Exception(f"Error reading file: {str(e)}")

def main():
    parser = argparse.ArgumentParser(description="Multi-Prompt Text Summarizer using Ollama API")

    input_group = parser.add_mutually_exclusive_group(required=True)
    input_group.add_argument('-t', '--text', type=str, help="Text to summarize")
    input_group.add_argument('-f', '--file', type=str, help="Path to text file to summarize")

    parser.add_argument('--url', type=str,
                       default="http://localhost:11434",
                       help="Ollama API base URL")
    parser.add_argument('--model', type=str,
                       default="qwen2:0.5b",
                       help="Model name to use")

    args = parser.parse_args()

    try:
        # Initialize summarizer
        summarizer = MultiPromptSummarizer(base_url=args.url, model=args.model)

        # Get content
        content = read_file(args.file) if args.file else args.text

        # Generate all summaries
        print("\nGenerating summaries using different approaches...")
        summaries = summarizer.summarize_all(content)

        # Print results
        print("\n" + "="*80)
        print("SUMMARY RESULTS")
        print("="*80)

        for prompt_name, summary in summaries.items():
            print(f"\n{prompt_name}")
            print("-" * len(prompt_name))
            print(summary)
            print("\n" + "="*80)

    except Exception as e:
        print(f"Error: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Writing multiprompt_summarizer.py


In [ ]:
!python summarizer.py -f /content/book.txt --model gemma2

Generating summary...

Summary:
--------------------------------------------------------------------------------
This excerpt from "Machine Learning Mastery" delves into the final chapters of the book, highlighting the reader's accomplishments and providing guidance for further learning.

**Key Points:**

* **Congratulations!** The author commends the reader for completing the journey through machine learning algorithms, emphasizing their significant progress in understanding and implementing these concepts.
* **Resources for Continued Learning:** The text offers recommendations for books, forums, Q&A websites, and direct contact with the author to facilitate ongoing learning and problem-solving in the field of machine learning.
* **Looking Ahead:**  The book concludes by encouraging readers to continue exploring and mastering machine learning algorithms, assuring them that this is just the beginning of their journey.

Overall, these final chapters serve as a celebration of the reader'

In [ ]:
!python summarizer.py -f /content/samsung_microwave_support.txt --model gemma2

Generating summary...

Summary:
--------------------------------------------------------------------------------
This guide outlines troubleshooting steps for Samsung microwaves that aren't heating properly. 

**Start by:**

* **Checking the power supply:** Ensure it's plugged into a working outlet and the cord isn't damaged.
* **Inspecting the door seal:** Make sure it's clean, intact, and the door is fully closed.

**Then try:**

* **Testing the cooking mode:** Heat a cup of water for 1-2 minutes on high power. If it doesn't heat, there might be an issue with internal components.
* **Resetting the microwave:** Unplug it for 30 seconds, then plug it back in and restart.

**If these steps don't work, contact Samsung Support for further assistance.**  Provide your microwave's model and serial number.
--------------------------------------------------------------------------------


In [ ]:
!python summarizer.py -f /content/samsung_refrigerator_support.txt --model gemma2

Generating summary...

Summary:
--------------------------------------------------------------------------------
This guide provides troubleshooting steps for cooling issues in Samsung refrigerators.  

**Steps to take:**

* **Check temperature settings:** Make sure both the fridge and freezer are set to the recommended temperatures (3°C/ -18°C).
* **Inspect door seals:** Ensure they are clean, undamaged, and creating a tight seal. 
* **Clear ventilation:** Remove any food blocking air vents inside the refrigerator.
* **Clean coils:** Use a vacuum or brush to clean the condenser coils located at the back or bottom of the fridge.
* **Power cycle:** Unplug the refrigerator for 5 minutes, then plug it back in.

If these steps don't resolve the issue, contact Samsung Support or schedule a repair.
--------------------------------------------------------------------------------


In [ ]:
!python summarizer.py -f /content/samsung_mobile_support.txt --model gemma2

Generating summary...

Summary:
--------------------------------------------------------------------------------
This guide outlines how to resolve battery drain issues on Samsung Galaxy smartphones. 

**Key tips include:** closing unused apps, optimizing settings (brightness, Bluetooth, GPS), enabling Battery Saver mode, checking app usage and uninstalling power-hungry ones, ensuring software updates are installed, and considering a battery replacement if the problem persists after two years.
--------------------------------------------------------------------------------


In [ ]:
!python multiprompt_summarizer.py -f /content/book.txt --model gemma2


Generating summaries using different approaches...

Generating TLDR Style summary...

Generating Structured Analysis summary...

Generating Concise 5-Point summary...

Generating Comprehensive TLDR summary...

SUMMARY RESULTS

TLDR Style
----------
The text you provided is a sample chapter from a book about machine learning algorithms. It outlines the journey of learning these algorithms and provides resources for further exploration.

Here's a breakdown of what the text covers:

* **Introduction:** The author congratulates the reader on completing the book and emphasizes the significant progress they've made in understanding machine learning algorithms.
* **Summary of Learning Journey:**  The text highlights key milestones achieved during the learning process, including grasping fundamental concepts, implementing various algorithms (linear regression, logistic regression, decision trees, support vector machines, etc.), and exploring ensemble methods like bagging and boosting.
* **Fur

In [ ]:
!python multiprompt_summarizer.py -f /content/samsung_microwave_support.txt --model gemma2


Generating summaries using different approaches...

Generating TLDR Style summary...

Generating Structured Analysis summary...

Generating Concise 5-Point summary...

Generating Comprehensive TLDR summary...

SUMMARY RESULTS

TLDR Style
----------
## TLDR: Samsung Microwave Not Heating?

* **Check Power:** Make sure it's plugged in properly and the cord isn't damaged.
* **Inspect Door Seal:**  Ensure the door is fully closed and the seals are clean.
* **Test Function:** Heat a cup of water for 1-2 minutes on high power. If it doesn't heat, there could be an internal issue.
* **Restart Microwave:** Unplug for 30 seconds, then plug back in.
* **Contact Samsung:** If nothing works, call Samsung support with your model and serial number.


Structured Analysis
-------------------
Here's a breakdown of the text into bullet points:

**Problem/Issue Addressed:**

*  The document addresses issues with Samsung microwave ovens that are not heating properly.

**Key Steps/Solutions:**

* **Inspec

In [ ]:
!python multiprompt_summarizer.py -f /content/samsung_refrigerator_support.txt --model gemma2


Generating summaries using different approaches...

Generating TLDR Style summary...

Generating Structured Analysis summary...

Generating Concise 5-Point summary...

Generating Comprehensive TLDR summary...

SUMMARY RESULTS

TLDR Style
----------
## TLDR; Samsung Fridge Not Cooling?

* **Check Temperature:** Is it set to 3°C for fridge and -18°C for freezer?
* **Inspect Seals:** Are door seals clean & undamaged? Replace if necessary.
* **Clear Ventilation:**  Ensure air vents inside the fridge are not blocked.
* **Clean Coils:** Vacuum or brush the condenser coils (back/bottom).
* **Power Cycle:** Unplug for 5 minutes, then plug back in.

**Still not cooling? Contact Samsung Support or schedule a repair.**


Structured Analysis
-------------------
Here's a breakdown of the troubleshooting guide:

**Problem/Issue Addressed:**  Samsung refrigerator is not cooling properly.

**Key Steps/Solutions:**

* **Check Temperature Settings:** Verify fridge and freezer are set to 3°C and -18°C r

In [ ]:
!python multiprompt_summarizer.py -f /content/samsung_tv_support.txt --model gemma2


Generating summaries using different approaches...

Generating TLDR Style summary...

Generating Structured Analysis summary...

Generating Concise 5-Point summary...

Generating Comprehensive TLDR summary...

SUMMARY RESULTS

TLDR Style
----------
## TLDR; Samsung Smart TV Connectivity Problems?

* **Check Network:** Make sure your router is working and other devices can connect. 
* **Restart Everything:** Power cycle both your TV and router.
* **Update Software:** Go to Settings > Support > Software Update and select "Update Now."
* **Factory Reset (Last Resort):**  Settings > General > Reset (PIN: 0000). Be warned, this erases all data!
* **Contact Samsung:** If nothing works, reach out with your TV model and a detailed problem description.


Structured Analysis
-------------------
Here's a breakdown of the text into bullet points:

**Problem/Issue Addressed:**  Connectivity problems with Samsung Smart TVs, preventing streaming, browsing, or other smart functions.

**Key Steps/Solu

In [ ]:
!curl http://localhost:11434/api/tags